<a href="https://colab.research.google.com/github/Bellladraf6/machine-learning-labs/blob/main/Exercise4_Question.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Ниже приведен код со ссылкой на набор данных «happy» или «sad», содержащий 80 изображений, 40 счастливых и 40 грустных.
Создайте сверточную нейронную сеть, которая обучается со 100% точностью на этих изображениях, с отменой обучения при достижении точности обучения .999

In [1]:
import tensorflow as tf
import os
import zipfile
#
!wget --no-check-certificate "https://github.com/andrew-veriga/Tensorflow-labs/raw/master/happy-or-sad.zip" \
   -O "/tmp/happy-or-sad.zip"


--2024-05-25 19:27:17--  https://github.com/andrew-veriga/Tensorflow-labs/raw/master/happy-or-sad.zip
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/andrew-veriga/Tensorflow-labs/master/happy-or-sad.zip [following]
--2024-05-25 19:27:18--  https://raw.githubusercontent.com/andrew-veriga/Tensorflow-labs/master/happy-or-sad.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2670333 (2.5M) [application/zip]
Saving to: ‘/tmp/happy-or-sad.zip’

/tmp/happy-or-sad.z 100%[===================>]   2.55M  --.-KB/s    in 0.07s   

2024-05-25 19:27:18 (35.3 MB/s) - ‘/tmp/happy-or-sad.zip’ saved [2670333/26

In [2]:
zip_ref = zipfile.ZipFile("/tmp/happy-or-sad.zip", 'r')
zip_ref.extractall("/tmp/h-or-s")
zip_ref.close()

import os

train_happy_dir = os.path.join('/tmp/h-or-s/happy')

train_sad_dir = os.path.join('/tmp/h-or-s/sad')

train_happy_names = os.listdir(train_happy_dir)
print(train_happy_names[:10])

train_sad_names = os.listdir(train_sad_dir)
print(train_sad_names[:10])

['happy2-13.png', 'happy1-11.png', 'happy1-15.png', 'happy1-12.png', 'happy2-06.png', 'happy2-11.png', 'happy1-01.png', 'happy2-05.png', 'happy1-17.png', 'happy2-16.png']
['sad1-17.png', 'sad1-16.png', 'sad2-01.png', 'sad1-07.png', 'sad1-18.png', 'sad2-03.png', 'sad2-13.png', 'sad2-00.png', 'sad1-04.png', 'sad1-08.png']


In [3]:

DESIRED_ACCURACY = 0.999

class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if logs.get('accuracy') > DESIRED_ACCURACY:
            print("\nДостигнута точность {}%, обучение прерывается!".format(DESIRED_ACCURACY*100))
            self.model.stop_training = True

callbacks = myCallback()

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

from tensorflow.keras.optimizers import RMSprop

model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(lr=0.001),
              metrics=['accuracy'])

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1/255)

train_generator = train_datagen.flow_from_directory(
        '/tmp/h-or-s/',
        target_size=(150, 150),
        batch_size=8,
        class_mode='binary')

history = model.fit(
      train_generator,
      steps_per_epoch=10,
      epochs=30,
      verbose=1,
      callbacks=[callbacks]
      )


Found 80 images belonging to 2 classes.
Epoch 1/30
10/10 [==============================] - 5s 357ms/step - loss: 1.4976 - accuracy: 0.5250
Epoch 2/30
10/10 [==============================] - 4s 343ms/step - loss: 0.5025 - accuracy: 0.7625
Epoch 3/30
10/10 [==============================] - 3s 266ms/step - loss: 0.3930 - accuracy: 0.8375
Epoch 4/30
10/10 [==============================] - 3s 267ms/step - loss: 0.1664 - accuracy: 0.9625
Epoch 5/30
10/10 [==============================] - 3s 277ms/step - loss: 0.1407 - accuracy: 0.9500
Epoch 6/30
10/10 [==============================] - 3s 265ms/step - loss: 0.1355 - accuracy: 0.9500
Epoch 7/30
10/10 [==============================] - 3s 265ms/step - loss: 0.0419 - accuracy: 0.9875
Epoch 8/30
10/10 [==============================] - 3s 272ms/step - loss: 0.0851 - accuracy: 0.9500
Epoch 9/30
10/10 [==============================] - ETA: 0s - loss: 0.0242 - accuracy: 1.0000
Достигнута точность 99.9%, обучение прерывается!
10/10 [==========